# Wikipedia training

### In this tutorial we will:
 - Learn how to train the NMF topic model on the English Wikipedia corpus
 - Compare it with LDA and Sklearn NMF
 - Evaluate results

In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import json
import logging
import numpy as np
import pandas as pd
import scipy.sparse
from smart_open import smart_open
import time
import os
import psutil
from contextlib import contextmanager
from multiprocessing import Process
from tqdm import tqdm, tqdm_notebook
import joblib

import gensim.downloader as api
from gensim import matutils
from gensim.corpora import MmCorpus, Dictionary
from gensim.models import LdaModel, CoherenceModel
from gensim.models.nmf import Nmf as GensimNmf
from sklearn.decomposition.nmf import NMF as SklearnNmf
from gensim.parsing.preprocessing import preprocess_string

tqdm.pandas()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

# Preprocessing

### Load wikipedia dump
Let's use `gensim.downloader.api` for that

In [2]:
data = api.load("wiki-english-20171001")
article = next(iter(data))

for section_title, section_text in zip(
    article['section_titles'],
    article['section_texts']
):
    print("Section title: %s" % section_title)
    print("Section text: %s" % section_text[:100])

Section title: Introduction
Section text: 




'''Anarchism''' is a political philosophy that advocates self-governed societies based on volun
Section title: Etymology and terminology
Section text: 

The word ''anarchism'' is composed from the word ''anarchy'' and the suffix ''-ism'', themselves d
Section title: History
Section text: 

===Origins===
Woodcut from a Diggers document by William Everard

The earliest anarchist themes ca
Section title: Anarchist schools of thought
Section text: 
Portrait of philosopher Pierre-Joseph Proudhon (1809–1865) by Gustave Courbet. Proudhon was the pri
Section title: Internal issues and debates
Section text: 
consistent with anarchist values is a controversial subject among anarchists.

Anarchism is a philo
Section title: Topics of interest
Section text: Intersecting and overlapping between various schools of thought, certain topics of interest and inte
Section title: Criticisms
Section text: 
Criticisms of anarchism include moral criticisms and pra

Preprocess and save articles

In [3]:
def save_preprocessed_articles(filename, articles):
    with smart_open(filename, 'w+', encoding="utf8") as writer:
        for article in tqdm_notebook(articles):
            article_text = " ".join(
                " ".join(section)
                for section
                in zip(
                    article['section_titles'],
                    article['section_texts']
                )
            )
            article_text = preprocess_string(article_text)

            writer.write(json.dumps(article_text) + '\n')


def get_preprocessed_articles(filename):
    with smart_open(filename, 'r', encoding="utf8") as reader:
        for line in tqdm_notebook(reader):
            yield json.loads(
                line
            )

In [4]:
SAVE_ARTICLES = False

if SAVE_ARTICLES:
    save_preprocessed_articles('wiki_articles.jsonlines', data)

### Create and save dictionary

In [5]:
SAVE_DICTIONARY = False

if SAVE_DICTIONARY:
    dictionary = Dictionary(get_preprocessed_articles('wiki_articles.jsonlines'))
    dictionary.save('wiki.dict')

### Load and filter dictionary

In [6]:
dictionary = Dictionary.load('wiki.dict')
dictionary.filter_extremes()
dictionary.compactify()

2019-02-06 22:27:02,588 : INFO : loading Dictionary object from wiki.dict
2019-02-06 22:27:03,510 : INFO : loaded wiki.dict
2019-02-06 22:27:09,054 : INFO : discarding 1910146 tokens: [('abdelrahim', 49), ('abstention', 120), ('anarcha', 101), ('anarchica', 40), ('anarchosyndicalist', 20), ('antimilitar', 68), ('arbet', 194), ('archo', 100), ('arkhē', 5), ('autonomedia', 118)]...
2019-02-06 22:27:09,055 : INFO : keeping 100000 tokens which were in no less than 5 and no more than 2462447 (=50.0%) documents
2019-02-06 22:27:09,956 : INFO : resulting dictionary: Dictionary(100000 unique tokens: ['tago', 'süß', 'akrita', 'divert', 'construccion']...)


### MmCorpus wrapper
In this way we'll:

- Make sure that documents are shuffled
- Be able to train-test split corpus without rewriting it

In [7]:
class RandomCorpus(MmCorpus):
    def __init__(self, random_seed=42, testset=False, testsize=1000, *args,
                 **kwargs):
        super().__init__(*args, **kwargs)

        random_state = np.random.RandomState(random_seed)
        
        self.indices = random_state.permutation(range(self.num_docs))
        test_nnz = sum(len(self[doc_idx]) for doc_idx in self.indices[:testsize])
        
        if testset:
            self.indices = self.indices[:testsize]
            self.num_docs = testsize
            self.num_nnz = test_nnz
        else:
            self.indices = self.indices[testsize:]
            self.num_docs -= testsize
            self.num_nnz -= test_nnz

    def __iter__(self):
        for doc_id in self.indices:
            yield self[doc_id]

### Create and save corpus

In [8]:
SAVE_CORPUS = False

if SAVE_CORPUS:
    corpus = (
        dictionary.doc2bow(article)
        for article
        in get_preprocessed_articles('wiki_articles.jsonlines')
    )
    
    RandomCorpus.serialize('wiki.mm', corpus)

### Load train and test corpus
Using `RandomCorpus` wrapper

In [9]:
train_corpus = RandomCorpus(
    random_seed=42, testset=False, testsize=2000, fname='wiki.mm'
)
test_corpus = RandomCorpus(
    random_seed=42, testset=True, testsize=2000, fname='wiki.mm'
)

2019-02-06 22:27:10,847 : INFO : loaded corpus index from wiki.mm.index
2019-02-06 22:27:10,847 : INFO : initializing cython corpus reader from wiki.mm
2019-02-06 22:27:10,848 : INFO : accepted corpus with 4924894 documents, 100000 features, 683326444 non-zero entries
2019-02-06 22:27:17,213 : INFO : loaded corpus index from wiki.mm.index
2019-02-06 22:27:17,215 : INFO : initializing cython corpus reader from wiki.mm
2019-02-06 22:27:17,218 : INFO : accepted corpus with 4924894 documents, 100000 features, 683326444 non-zero entries


### Convert corpora to csc and save

It's necessary in order to train Sklearn NMF. These matrices take **a lot** of RAM even though they're sparse (about 8GB).

In [10]:
SAVE_CSC = False

if SAVE_CSC:
    train_csc = matutils.corpus2csc(train_corpus, len(dictionary))
    scipy.sparse.save_npz('train_csc.npz', train_csc)
    
    test_csc = matutils.corpus2csc(test_corpus, len(dictionary))
    scipy.sparse.save_npz('test_csc.npz', test_csc)

## Metrics

- `train time` - time to train a model
- `mean_ram` - mean RAM consumption during the training
- `max_ram` - maximum RAM consumpiton during the training
- `perplexity` - perplexity score. Another usual TM metric
- `coherence` - coherence score (not defined for sklearn NMF). Classic metric for topic models.
- `l2_norm` - l2 norm of `v - Wh`

In [11]:
@contextmanager
def measure_ram(output, tick=2):
    def _measure_ram(pid, output, start_memory, tick=5):
        py = psutil.Process(pid)
        with open(output, 'w') as outfile:
            while True:
                memory = py.memory_info().rss - start_memory
                outfile.write("{}\n".format(memory))
                outfile.flush()
                time.sleep(tick)

    pid = os.getpid()
    start_memory = psutil.Process(pid).memory_info().rss
    p = Process(target=_measure_ram, args=(pid, output, start_memory, 5))
    p.start()
    yield
    p.terminate()

def get_train_time_and_ram(func, name):
    memprof_filename = "{}.memprof".format(name)
    
    start = time.time()

    with measure_ram(memprof_filename, 5):
        result = func()        
        
    elapsed_time = pd.to_timedelta(time.time() - start, unit='s').round('s')
    
    memprof_df = pd.read_csv(memprof_filename, squeeze=True)
    
    mean_ram = "{} MB".format(
        int(memprof_df.mean() // 2**20),
    )
    
    max_ram = "{} MB".format(int(memprof_df.max() // 2**20))

    return elapsed_time, mean_ram, max_ram, result


def get_tm_metrics(model, test_corpus):
    W = model.get_topics().T
    H = np.zeros((model.num_topics, len(test_corpus)))
    for bow_id, bow in enumerate(test_corpus):
        for topic_id, word_count in model.get_document_topics(bow):
            H[topic_id, bow_id] = word_count

    pred_factors = W.dot(H)
    
    dense_corpus = matutils.corpus2dense(test_corpus, pred_factors.shape[0])

    l2_norm = get_tm_l2_norm(pred_factors, dense_corpus)
    
    pred_factors /= pred_factors.sum(axis=0)
    
    perplexity = get_tm_perplexity(pred_factors, dense_corpus)

    model.normalize = True

    coherence = CoherenceModel(
        model=model,
        corpus=test_corpus,
        coherence='u_mass'
    ).get_coherence()
    
    topics = model.show_topics(5)

    model.normalize = False

    return dict(
        perplexity=round(perplexity, 4),
        coherence=round(coherence, 4),
        l2_norm=round(l2_norm, 4),
        topics=topics,
    )


def get_tm_perplexity(pred_factors, dense_corpus):
    return np.exp(-(np.log(pred_factors, where=pred_factors > 0) * dense_corpus).sum() / dense_corpus.sum())


def get_tm_l2_norm(pred_factors, dense_corpus):
    return np.linalg.norm(dense_corpus - pred_factors)


def get_sklearn_topics(model, id2word, top_n=5):
    topic_probas = model.components_.T
    topic_probas = topic_probas / topic_probas.sum(axis=0)
    
    sparsity = np.zeros(topic_probas.shape[1])

    for row in topic_probas:
        sparsity += (row == 0)

    sparsity /= topic_probas.shape[1]
    
    topic_probas = topic_probas[:, sparsity.argsort()[::-1]][:, :top_n]
    
    token_indices = topic_probas.argsort(axis=0)[:-11:-1, :]
    topic_probas.sort(axis=0)
    topic_probas = topic_probas[:-11:-1, :]
    
    topics = []
    
    for topic_idx in range(topic_probas.shape[1]):
        tokens = [
            id2word[token_idx]
            for token_idx
            in token_indices[:, topic_idx]
        ]
        topic = (
            '{}*"{}"'.format(round(proba, 3), token)
            for proba, token
            in zip(topic_probas[:, topic_idx], tokens)
        )
        topic = " + ".join(topic)
        topics.append((topic_idx, topic))
    
    return topics


def get_sklearn_metrics(model, test_corpus, dictionary):
    W = model.components_.T
    H = model.transform((test_corpus).T).T
    pred_factors = W.dot(H)
    
    l2_norm = np.linalg.norm(test_corpus - pred_factors)
    
    pred_factors /= pred_factors.sum(axis=0)

    perplexity = np.exp(
        -(np.log(pred_factors, where=pred_factors > 0) * test_corpus).sum()
        / test_corpus.sum()
    )
    
    topics = get_sklearn_topics(model, dictionary, top_n=5)

    return dict(
        perplexity=perplexity,
        l2_norm=l2_norm,
        topics=topics,
    )

Define the dataframe in which we'll store metrics

In [12]:
tm_metrics = pd.DataFrame(columns=[
    'model', 'train_time', 'mean_ram', 'max_ram', 'perplexity', 'coherence', 'l2_norm', 'topics'
])

### Define common params for the models

In [13]:
params = dict(
    corpus=train_corpus,
    chunksize=2000,
    num_topics=50,
    id2word=dictionary,
    passes=1,
    eval_every=10,
    minimum_probability=0,
    random_state=42,
)

## Training

### Train Gensim NMF and save it
Normalization is turned off to compute metrics correctly

In [14]:
row = dict()
row['model'] = 'gensim_nmf'
row['train_time'], row['mean_ram'], row['max_ram'], nmf = get_train_time_and_ram(
    lambda: GensimNmf(
        normalize=False,
        **params
    ),
    'gensim_nmf',
)

nmf.save('gensim_nmf.model')

2019-02-06 22:28:18,836 : INFO : Loss: 0.04683116478141604
2019-02-06 22:28:47,687 : INFO : Loss: 0.03585093916474252
2019-02-06 22:29:03,704 : INFO : Loss: 0.016218137567651437
2019-02-06 22:29:21,170 : INFO : Loss: 0.014643892581399755
2019-02-06 22:29:39,115 : INFO : Loss: 0.0227215203863265
2019-02-06 22:29:55,297 : INFO : Loss: 0.011330135672850988
2019-02-06 22:30:06,873 : INFO : Loss: 0.006364371284778088
2019-02-06 22:30:22,292 : INFO : Loss: 0.005515037915443601
2019-02-06 22:30:31,774 : INFO : Loss: 0.00514751254979705
2019-02-06 22:30:41,246 : INFO : Loss: 0.004481632057458362
2019-02-06 22:30:50,361 : INFO : Loss: 0.0039307233697034455
2019-02-06 22:30:59,781 : INFO : Loss: 0.005137617660878855
2019-02-06 22:31:07,843 : INFO : Loss: 0.003985348592745008
2019-02-06 22:31:22,565 : INFO : Loss: 0.004099755242047697
2019-02-06 22:31:31,386 : INFO : Loss: 0.0013553803901884735
2019-02-06 22:31:38,630 : INFO : Loss: 0.0029466572709945004
2019-02-06 22:31:46,466 : INFO : Loss: 0.0

2019-02-06 22:43:49,286 : INFO : Loss: 0.00024350635516826678
2019-02-06 22:43:54,347 : INFO : Loss: 0.00021107625583640768
2019-02-06 22:43:59,144 : INFO : Loss: 0.0
2019-02-06 22:44:04,209 : INFO : Loss: 0.0
2019-02-06 22:44:09,360 : INFO : Loss: 0.00019698529746700524
2019-02-06 22:44:14,517 : INFO : Loss: 0.0005681332506493842
2019-02-06 22:44:19,699 : INFO : Loss: 0.00014944867864539842
2019-02-06 22:44:24,466 : INFO : Loss: 0.00035950934816992203
2019-02-06 22:44:29,360 : INFO : Loss: 0.0004279386451412483
2019-02-06 22:44:34,231 : INFO : Loss: 0.0
2019-02-06 22:44:39,234 : INFO : Loss: 0.00015970718295116123
2019-02-06 22:44:44,264 : INFO : Loss: 0.0003263379130256862
2019-02-06 22:44:49,299 : INFO : Loss: 0.00011964296446390053
2019-02-06 22:44:54,428 : INFO : Loss: 0.00022925677738528728
2019-02-06 22:44:59,317 : INFO : Loss: 0.00020768296825596713
2019-02-06 22:45:04,230 : INFO : Loss: 0.0003238046557707154
2019-02-06 22:45:09,382 : INFO : Loss: 0.00026031612396985496
2019-02

### Load Gensim NMF and store metrics

In [15]:
nmf = GensimNmf.load('gensim_nmf.model')
row.update(get_tm_metrics(nmf, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

2019-02-06 22:52:36,983 : INFO : loading Nmf object from gensim_nmf.model
2019-02-06 22:52:37,292 : INFO : loading id2word recursively from gensim_nmf.model.id2word.* with mmap=None
2019-02-06 22:52:37,293 : INFO : loaded gensim_nmf.model
/home/anotherbugmaster/gensim/gensim/matutils.py:503: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)
2019-02-06 22:53:31,168 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2019-02-06 22:53:31,278 : INFO : CorpusAccumulator accumulated stats from 2000 documents


### Train LDA and save it
That's a common model to do Topic Modeling

In [16]:
row = dict()
row['model'] = 'lda'
row['train_time'], row['mean_ram'], row['max_ram'], lda = get_train_time_and_ram(
    lambda: LdaModel(**params),
    'lda',
)
lda.save('lda.model')

2019-02-06 22:53:31,779 : INFO : using symmetric alpha at 0.02
2019-02-06 22:53:31,780 : INFO : using symmetric eta at 0.02
2019-02-06 22:53:31,798 : INFO : using serial LDA version on this node
2019-02-06 22:53:32,320 : INFO : running online (single-pass) LDA training, 50 topics, 1 passes over the supplied corpus of 4922894 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-02-06 22:53:32,473 : INFO : PROGRESS: pass 0, at document #2000/4922894
2019-02-06 22:53:34,391 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-06 22:53:34,709 : INFO : topic #36 (0.020): 0.006*"new" + 0.005*"record" + 0.004*"servic" + 0.004*"includ" + 0.003*"year" + 0.003*"state" + 0.003*"design" + 0.003*"work" + 0.003*"time" + 0.003*"american"
2019-02-06 22:53:34,711 : INFO : topic #19 (0.020): 0.005*"team" + 0.005*"new" + 0.004*"nation" + 0.004*"state" + 0.004*"includ"

2019-02-06 22:53:45,703 : INFO : topic diff=0.220294, rho=0.408248
2019-02-06 22:53:45,936 : INFO : PROGRESS: pass 0, at document #14000/4922894
2019-02-06 22:53:47,587 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-06 22:53:47,846 : INFO : topic #39 (0.020): 0.038*"team" + 0.025*"season" + 0.025*"coach" + 0.020*"leagu" + 0.015*"plai" + 0.012*"playoff" + 0.012*"footbal" + 0.010*"hockei" + 0.009*"year" + 0.009*"batavia"
2019-02-06 22:53:47,847 : INFO : topic #0 (0.020): 0.035*"station" + 0.031*"art" + 0.015*"line" + 0.014*"railwai" + 0.012*"museum" + 0.012*"paint" + 0.012*"left" + 0.011*"align" + 0.010*"work" + 0.009*"artist"
2019-02-06 22:53:47,848 : INFO : topic #13 (0.020): 0.020*"south" + 0.018*"north" + 0.015*"west" + 0.013*"east" + 0.012*"catalana" + 0.012*"lake" + 0.010*"counti" + 0.009*"villag" + 0.008*"vallei" + 0.008*"hospit"
2019-02-06 22:53:47,849 : INFO : topic #45 (0.020): 0.015*"design" + 0.007*"snake" + 0.006*"standard" + 0.006*"de

2019-02-06 22:54:04,124 : INFO : topic #10 (0.020): 0.010*"cell" + 0.009*"diseas" + 0.007*"carson" + 0.007*"gene" + 0.007*"acupunctur" + 0.007*"caus" + 0.006*"patient" + 0.006*"treatment" + 0.006*"protein" + 0.005*"includ"
2019-02-06 22:54:04,126 : INFO : topic #18 (0.020): 0.007*"man" + 0.006*"kill" + 0.005*"charact" + 0.005*"time" + 0.004*"seri" + 0.004*"later" + 0.004*"appear" + 0.004*"episod" + 0.004*"stori" + 0.004*"father"
2019-02-06 22:54:04,126 : INFO : topic #0 (0.020): 0.055*"station" + 0.038*"art" + 0.024*"line" + 0.024*"railwai" + 0.016*"paint" + 0.016*"work" + 0.013*"align" + 0.012*"train" + 0.012*"museum" + 0.012*"london"
2019-02-06 22:54:04,127 : INFO : topic #37 (0.020): 0.007*"love" + 0.007*"time" + 0.006*"plai" + 0.006*"releas" + 0.005*"appear" + 0.005*"night" + 0.004*"man" + 0.004*"song" + 0.003*"girl" + 0.003*"year"
2019-02-06 22:54:04,129 : INFO : topic #35 (0.020): 0.022*"russian" + 0.020*"russia" + 0.018*"parti" + 0.017*"philippin" + 0.013*"union" + 0.013*"moscow

2019-02-06 22:54:15,919 : INFO : topic #25 (0.020): 0.016*"mountain" + 0.015*"river" + 0.015*"park" + 0.014*"lake" + 0.013*"mount" + 0.011*"area" + 0.008*"peak" + 0.007*"rock" + 0.006*"natur" + 0.006*"survei"
2019-02-06 22:54:15,920 : INFO : topic #27 (0.020): 0.045*"race" + 0.015*"dai" + 0.012*"team" + 0.012*"tour" + 0.012*"time" + 0.010*"rider" + 0.010*"hors" + 0.010*"stage" + 0.010*"finish" + 0.009*"second"
2019-02-06 22:54:15,922 : INFO : topic #48 (0.020): 0.066*"octob" + 0.062*"januari" + 0.061*"march" + 0.057*"septemb" + 0.055*"novemb" + 0.053*"june" + 0.053*"april" + 0.052*"juli" + 0.051*"august" + 0.051*"decemb"
2019-02-06 22:54:15,928 : INFO : topic diff=0.286263, rho=0.229416
2019-02-06 22:54:18,759 : INFO : -11.672 per-word bound, 3262.6 perplexity estimate based on a held-out corpus of 2000 documents with 564313 words
2019-02-06 22:54:18,760 : INFO : PROGRESS: pass 0, at document #40000/4922894
2019-02-06 22:54:20,304 : INFO : merging changes from 2000 documents into a mod

2019-02-06 22:54:30,428 : INFO : topic #21 (0.020): 0.025*"san" + 0.021*"spanish" + 0.019*"mexico" + 0.016*"del" + 0.012*"judi" + 0.012*"spain" + 0.012*"mexican" + 0.011*"puerto" + 0.010*"saturdai" + 0.010*"junkyard"
2019-02-06 22:54:30,429 : INFO : topic #49 (0.020): 0.028*"ingli" + 0.022*"indian" + 0.007*"slime" + 0.006*"peopl" + 0.006*"partit" + 0.006*"tropic" + 0.006*"ali" + 0.006*"precondit" + 0.006*"area" + 0.006*"khan"
2019-02-06 22:54:30,435 : INFO : topic diff=0.237375, rho=0.200000
2019-02-06 22:54:30,587 : INFO : PROGRESS: pass 0, at document #52000/4922894
2019-02-06 22:54:32,118 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-06 22:54:32,383 : INFO : topic #21 (0.020): 0.027*"san" + 0.020*"spanish" + 0.019*"mexico" + 0.018*"del" + 0.013*"judi" + 0.011*"mexican" + 0.011*"spain" + 0.011*"junkyard" + 0.010*"puerto" + 0.010*"rune"
2019-02-06 22:54:32,384 : INFO : topic #25 (0.020): 0.022*"river" + 0.015*"lake" + 0.015*"mountain" + 0.012*"

...

2019-02-07 00:06:23,360 : INFO : topic #30 (0.020): 0.036*"leagu" + 0.036*"club" + 0.029*"plai" + 0.027*"team" + 0.024*"season" + 0.024*"cup" + 0.022*"footbal" + 0.017*"goal" + 0.016*"match" + 0.012*"score"
2019-02-07 00:06:23,366 : INFO : topic diff=0.002811, rho=0.021890
2019-02-07 00:06:23,521 : INFO : PROGRESS: pass 0, at document #4176000/4922894
2019-02-07 00:06:24,878 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-07 00:06:25,144 : INFO : topic #32 (0.020): 0.049*"district" + 0.045*"popul" + 0.043*"villag" + 0.036*"town" + 0.033*"counti" + 0.023*"municip" + 0.022*"ag" + 0.021*"area" + 0.019*"censu" + 0.018*"citi"
2019-02-07 00:06:25,145 : INFO : topic #41 (0.020): 0.041*"citi" + 0.023*"park" + 0.020*"new" + 0.017*"street" + 0.013*"open" + 0.013*"center" + 0.011*"includ" + 0.011*"locat" + 0.010*"dai" + 0.009*"hotel"
2019-02-07 00:06:25,146 : INFO : topic #24 (0.020): 0.040*"book" + 0.035*"publish" + 0.027*"work" + 0.020*"new" + 0.014*"edit"

2019-02-07 00:06:36,822 : INFO : topic diff=0.003025, rho=0.021858
2019-02-07 00:06:36,981 : INFO : PROGRESS: pass 0, at document #4188000/4922894
2019-02-07 00:06:38,365 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-07 00:06:38,632 : INFO : topic #27 (0.020): 0.075*"race" + 0.021*"car" + 0.018*"team" + 0.014*"tour" + 0.013*"rey" + 0.012*"hors" + 0.012*"drunken" + 0.011*"ford" + 0.011*"finish" + 0.010*"time"
2019-02-07 00:06:38,633 : INFO : topic #1 (0.020): 0.056*"chinatown" + 0.046*"chines" + 0.026*"korda" + 0.025*"indi" + 0.021*"kunm" + 0.018*"korean" + 0.017*"lee" + 0.015*"singapor" + 0.015*"south" + 0.014*"kirk"
2019-02-07 00:06:38,633 : INFO : topic #26 (0.020): 0.030*"world" + 0.028*"championship" + 0.026*"women" + 0.024*"omiss" + 0.023*"men" + 0.022*"melodrama" + 0.019*"event" + 0.018*"team" + 0.018*"nation" + 0.017*"rank"
2019-02-07 00:06:38,634 : INFO : topic #42 (0.020): 0.047*"german" + 0.032*"germani" + 0.016*"von" + 0.014*"jewish" 

2019-02-07 00:06:51,731 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-07 00:06:51,998 : INFO : topic #9 (0.020): 0.073*"born" + 0.042*"american" + 0.028*"van" + 0.019*"dynasti" + 0.018*"playoff" + 0.017*"footbal" + 0.017*"politician" + 0.016*"english" + 0.014*"actor" + 0.013*"singer"
2019-02-07 00:06:51,999 : INFO : topic #39 (0.020): 0.061*"canada" + 0.046*"cheat" + 0.024*"hockei" + 0.024*"unlaw" + 0.020*"outpost" + 0.015*"montreal" + 0.015*"new" + 0.015*"ic" + 0.014*"shelton" + 0.014*"nováček"
2019-02-07 00:06:52,000 : INFO : topic #47 (0.020): 0.063*"music" + 0.034*"perform" + 0.021*"theatr" + 0.018*"compos" + 0.018*"plai" + 0.016*"danc" + 0.013*"opera" + 0.013*"orchestra" + 0.012*"piano" + 0.011*"work"
2019-02-07 00:06:52,001 : INFO : topic #20 (0.020): 0.140*"school" + 0.040*"student" + 0.033*"high" + 0.030*"educ" + 0.023*"colleg" + 0.018*"year" + 0.013*"program" + 0.010*"district" + 0.009*"grade" + 0.009*"teacher"
2019-02-07 00:06:52,002 :

2019-02-07 00:07:02,953 : INFO : topic #12 (0.020): 0.008*"number" + 0.007*"function" + 0.006*"point" + 0.006*"exampl" + 0.006*"gener" + 0.006*"space" + 0.006*"valu" + 0.006*"set" + 0.006*"time" + 0.006*"field"
2019-02-07 00:07:02,953 : INFO : topic #15 (0.020): 0.011*"social" + 0.010*"organ" + 0.010*"develop" + 0.009*"commun" + 0.009*"work" + 0.009*"group" + 0.008*"peopl" + 0.007*"human" + 0.007*"women" + 0.006*"support"
2019-02-07 00:07:02,954 : INFO : topic #9 (0.020): 0.071*"born" + 0.042*"american" + 0.027*"van" + 0.018*"dynasti" + 0.018*"playoff" + 0.017*"footbal" + 0.016*"politician" + 0.016*"english" + 0.013*"actor" + 0.013*"singer"
2019-02-07 00:07:02,955 : INFO : topic #39 (0.020): 0.060*"canada" + 0.045*"cheat" + 0.024*"hockei" + 0.023*"unlaw" + 0.021*"outpost" + 0.015*"montreal" + 0.015*"new" + 0.015*"ic" + 0.014*"shelton" + 0.014*"nováček"
2019-02-07 00:07:02,961 : INFO : topic diff=0.003219, rho=0.021791
2019-02-07 00:07:03,118 : INFO : PROGRESS: pass 0, at document #4214

2019-02-07 00:07:16,393 : INFO : topic #14 (0.020): 0.024*"forc" + 0.022*"air" + 0.021*"war" + 0.020*"armi" + 0.018*"command" + 0.014*"unit" + 0.013*"militari" + 0.013*"oper" + 0.013*"aircraft" + 0.011*"divis"
2019-02-07 00:07:16,394 : INFO : topic #31 (0.020): 0.051*"game" + 0.027*"season" + 0.025*"team" + 0.022*"playoff" + 0.019*"plai" + 0.014*"coach" + 0.013*"leagu" + 0.011*"footbal" + 0.010*"year" + 0.009*"record"
2019-02-07 00:07:16,395 : INFO : topic #48 (0.020): 0.081*"march" + 0.079*"septemb" + 0.079*"octob" + 0.073*"juli" + 0.072*"januari" + 0.071*"august" + 0.070*"june" + 0.070*"novemb" + 0.068*"april" + 0.066*"decemb"
2019-02-07 00:07:16,401 : INFO : topic diff=0.002320, rho=0.021760
2019-02-07 00:07:16,559 : INFO : PROGRESS: pass 0, at document #4226000/4922894
2019-02-07 00:07:17,951 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-07 00:07:18,217 : INFO : topic #45 (0.020): 0.044*"art" + 0.030*"judah" + 0.028*"file" + 0.027*"museum" +

### Load LDA and store metrics

In [17]:
lda = LdaModel.load('lda.model')
row.update(get_tm_metrics(lda, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

2019-02-07 00:19:19,731 : INFO : loading LdaModel object from lda.model
2019-02-07 00:19:19,734 : INFO : loading expElogbeta from lda.model.expElogbeta.npy with mmap=None
2019-02-07 00:19:19,740 : INFO : setting ignored attribute id2word to None
2019-02-07 00:19:19,741 : INFO : setting ignored attribute dispatcher to None
2019-02-07 00:19:19,742 : INFO : setting ignored attribute state to None
2019-02-07 00:19:19,743 : INFO : loaded lda.model
2019-02-07 00:19:19,743 : INFO : loading LdaState object from lda.model.state
2019-02-07 00:19:19,837 : INFO : loaded lda.model.state
/home/anotherbugmaster/gensim/gensim/matutils.py:503: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)
2019-02-07 00:19:41,681 : INFO : CorpusAccumulator accumulated stats from 100

### Train Sklearn NMF and store metrics

In [18]:
row = dict()
row['model'] = 'sklearn_nmf'
sklearn_nmf = SklearnNmf(n_components=50, tol=1e-2, random_state=42)
row['train_time'], row['mean_ram'], row['max_ram'], sklearn_nmf = get_train_time_and_ram(
    lambda: sklearn_nmf.fit(scipy.sparse.load_npz('train_csc.npz').T),
    'sklearn_nmf',
)

joblib.dump(sklearn_nmf, 'sklearn_nmf.joblib')

['sklearn_nmf.joblib']

### Load Sklearn NMF and store metrics

In [19]:
sklearn_nmf = joblib.load('sklearn_nmf.joblib')
row.update(get_sklearn_metrics(
    sklearn_nmf, scipy.sparse.load_npz('test_csc.npz').toarray(), dictionary
))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

In [20]:
tm_metrics.replace(np.nan, '-', inplace=True)

## Results

In [21]:
tm_metrics.drop('topics', axis=1)

,model,train_time,mean_ram,max_ram,perplexity,coherence,l2_norm
0,gensim_nmf,00:25:16,130 MB,165 MB,3741.860600,-2.9857,1983.378700
1,lda,01:25:48,140 MB,140 MB,4701.976000,-2.5286,2273.643000
2,sklearn_nmf,00:49:29,10716 MB,15961 MB,3943.036299,-,1987.280856


### Insights

Gensim NMF is better than Sklearn NMF in every aspect:

- **2x** faster


- Uses **80x-120x** less memory.

    About **8GB** of RAM comes from the input corpus sparse matrices, which, in contrast to Gensim NMF, can't be passed iteratively. But even if we forget about tremendous corpus size, Sklearn NMF still uses about **2-8 GB** of RAM, which is much larger than that of Gensim NMF and LDA.


- Still achieves better l2 norm and perplexity

Comparing to LDA, Gensim NMF is also better in almost everything:

- **3.5x** faster
- Achieves much better l2 norm and perplexity

Coherence is less than LDA's though.

### Topics

In [22]:
def compare_topics(tm_metrics):
    for _, row in tm_metrics.iterrows():
        print('\n{}:'.format(row.model))
        print("\n".join(str(topic) for topic in row.topics))
        
compare_topics(tm_metrics)


gensim_nmf:
(23, '0.007*"seri" + 0.006*"episod" + 0.006*"time" + 0.006*"appear" + 0.005*"later" + 0.005*"charact" + 0.005*"kill" + 0.005*"man" + 0.004*"work" + 0.004*"book"')
(19, '0.018*"king" + 0.012*"centuri" + 0.009*"church" + 0.007*"son" + 0.005*"princ" + 0.005*"french" + 0.005*"franc" + 0.005*"england" + 0.005*"kingdom" + 0.005*"year"')
(49, '0.079*"royal" + 0.038*"corp" + 0.034*"armi" + 0.028*"regiment" + 0.027*"capt" + 0.025*"townhous" + 0.020*"maj" + 0.020*"artilleri" + 0.017*"servic" + 0.017*"col"')
(38, '0.132*"mount" + 0.130*"iatrogen" + 0.128*"peak" + 0.126*"knightsbridg" + 0.126*"somedai" + 0.065*"survei" + 0.027*"octob" + 0.023*"campo" + 0.023*"css" + 0.018*"septemb"')
(43, '0.197*"linear" + 0.196*"secundaria" + 0.039*"newman" + 0.037*"septemb" + 0.034*"parallax" + 0.027*"octob" + 0.023*"knightsbridg" + 0.023*"anderson" + 0.023*"montana" + 0.023*"lanc"')

lda:
(35, '0.054*"russian" + 0.039*"soviet" + 0.032*"russia" + 0.027*"polish" + 0.026*"republ" + 0.025*"philippin" +

Seems like all models have successfully learned the topic representation of the corpus.